# Korrelsajonsanalyse og lineærregresjon Equinor, Oljepris og NOK

In [2]:
import pandas as pd
import statsmodels.api as sm
from scipy.stats import pearsonr

# Load the datasets
commodity_prices = pd.read_excel('Data/Processed/DAILY_COMMODITY_PRICES.xlsx')
exchange_rates = pd.read_excel('Data/Processed/DAILY_EXCHANGE_RATES.xlsx')
stock_prices = pd.read_excel('Data/Processed/DAILY_STOCK_PRICES.xlsx')

# Ensure the date is in the correct datetime format
stock_prices['Date'] = pd.to_datetime(stock_prices['Date'])
commodity_prices['Date'] = pd.to_datetime(commodity_prices['Date'])
exchange_rates['Date'] = pd.to_datetime(exchange_rates['Date'])

# Calculate the mean exchange rate for NOK against important trading currencies
necessary_currencies = ['USDNOK', 'EURNOK', 'CHFNOK', 'GBPNOK']
exchange_rates_filtered = exchange_rates[exchange_rates['Ticker'].isin(necessary_currencies)]
mean_nok_rates = exchange_rates_filtered.groupby('Date')['Mid'].mean().reset_index()
mean_nok_rates.rename(columns={'Mid': 'MeanNOKRate'}, inplace=True)

# Select the oil price data
oil_data = commodity_prices[commodity_prices['Ticker'] == 'OILDB-FDS'][['Date', 'Price']]
oil_data.rename(columns={'Price': 'OilPrice'}, inplace=True)

# Merge all data on the 'Date' column
data = pd.merge(stock_prices, oil_data, on='Date', how='inner')
data = pd.merge(data, mean_nok_rates, on='Date', how='inner')

# Filter data for a specific energy company, e.g., 'EQNR-NO'
company_data = data[data['Ticker'] == 'EQNR-NO']

# Drop any rows with missing data
company_data.dropna(inplace=True)

# Perform correlation analysis
corr_oil_price = pearsonr(company_data['Price'], company_data['OilPrice'])[0]
corr_nok_rate = pearsonr(company_data['Price'], company_data['MeanNOKRate'])[0]
print(f"Correlation between Stock Price and Oil Price: {corr_oil_price}")
print(f"Correlation between Stock Price and Mean NOK Rate: {corr_nok_rate}")

# Linear Regression: Stock Price as a function of Oil Price and Mean NOK Rate
X = company_data[['OilPrice', 'MeanNOKRate']]
X = sm.add_constant(X)  # adding a constant
Y = company_data['Price']

model = sm.OLS(Y, X).fit()
print(model.summary())


Correlation between Stock Price and Oil Price: 0.9233844104127353
Correlation between Stock Price and Mean NOK Rate: 0.5309431509661086
                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.861
Model:                            OLS   Adj. R-squared:                  0.861
Method:                 Least Squares   F-statistic:                     5408.
Date:                Tue, 25 Mar 2025   Prob (F-statistic):               0.00
Time:                        18:05:32   Log-Likelihood:                -8287.0
No. Observations:                1750   AIC:                         1.658e+04
Df Residuals:                    1747   BIC:                         1.660e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0

/var/folders/cj/0sj08jds0671hy9qb1nk7shw0000gn/T/ipykernel_5936/10104154.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  company_data.dropna(inplace=True)


Impact of Oil Prices and NOK Exchange Rate on Energy Sector Stock Prices
Our analysis investigated the effects of fluctuations in oil prices and the Norwegian Krone (NOK) exchange rate on the stock prices of key energy sector companies listed on the Oslo Stock Exchange. By integrating daily data on oil prices, NOK exchange rates, and stock prices, we performed both correlation analysis and linear regression to quantify these relationships.

Correlation Findings: The correlation analysis revealed a positive correlation between oil prices and the stock prices of Equinor ASA (EQNR-NO), suggesting that as oil prices increase, so too do the stock prices of Equinor. This relationship highlights the dependency of energy companies on commodity prices, reflecting broader industry trends where oil price movements significantly influence market valuations of energy firms. Conversely, the correlation between the NOK exchange rate and Equinor's stock prices was less pronounced, indicating that while exchange rate fluctuations impact financial performance, their effect is comparatively muted against the backdrop of global oil price dynamics.

Regression Analysis: The linear regression model further quantified these effects, showing statistically significant coefficients for both oil prices and NOK exchange rates. The model indicated that oil prices have a more substantial and statistically significant impact on stock prices than exchange rate fluctuations. This finding underscores the critical role that commodity prices play in the financial health and stock performance of energy companies. The regression coefficients provide a clear measure of how a unit change in oil prices or NOK exchange rates is expected to influence the stock prices, offering valuable insights for investors and analysts.

Implications for Stakeholders: For investors, understanding these dynamics is crucial for making informed investment decisions, particularly in a volatile energy market influenced by both geopolitical factors and currency fluctuations. Energy companies and policymakers can also benefit from these insights by better anticipating market responses to changes in oil prices and exchange rates, thus enhancing strategic planning and risk management.

Limitations and Further Research: While the analysis provides significant insights, it is subject to the inherent limitations of historical data and the assumption that past patterns will persist. Future research could expand to include more granular data, consider additional variables such as geopolitical events or new energy policies, and apply more complex models to capture the nonlinear dynamics of the markets.